In [6]:
from datetime import datetime
import requests, json, os
from typing import Dict, Annotated, Optional, List, Union
from dotenv import load_dotenv
load_dotenv()

from services.twilio import client
from services.cache import get_agent_metadata, get_all_agents
from services.chat.chat import similarity_search, get_embedding
from services.db.supabase_services import supabase_client
supabase = supabase_client()

from app.core.config import settings

### Auth

In [2]:
from composio import ComposioToolSet, App

toolset = ComposioToolSet(entity_id="Jessica")
entity = toolset.get_entity()
request = entity.initiate_connection(App.NOTION)

print(f"Open this URL to authenticate: {request.redirectUrl}")


[2024-11-23 04:31:41,237][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


Open this URL to authenticate: https://api.notion.com/v1/oauth/authorize?client_id=17eb9b75-7ad0-4fc1-b01d-12bb09271484&redirect_uri=https%3A%2F%2Fbackend.composio.dev%2Fapi%2Fv1%2Fauth-apps%2Fadd&scope=&response_type=code&owner=user&state=production_eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjb25uZWN0aW9uSWQiOiJjM2Q0MmIwOC1kMGJiLTRkMjUtOTBhNS01MDQyODFkMjBlMGIiLCJpbnRlZ3JhdGlvbklkIjoiZjk3MjE3MzctYWNjOC00MmY5LWFlMTgtZjk5NzM3MGFiZjQ3IiwiYXBwTmFtZSI6Im5vdGlvbiIsImNsaWVudEluZm8iOnsiY2xpZW50SWQiOiIwMTI3MTQ1OS04NjNmLTQ3NzAtYThhMy0xNjZhY2I3ODFmMzciLCJtZW1iZXJJZCI6IjJiZDI2MDVkLTJiOTgtNDBlOC1iYzU2LWY4ZDY2YTE1NjllYyJ9LCJpYXQiOjE3MzIzMzYzMDJ9.kT9d1rvKVDIFuU8b3VhMGyJRPi1wLCF2pg_XukZ7s78&code_challenge=cbF96qJ_f_MgSrGdoC-43i5ObxfQXi9TNCcmV5qBqJk&code_challenge_method=S256


In [3]:
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="Jessica")


[2024-11-23 04:32:28,947][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


### Notion

In [16]:
today = datetime.now().strftime("%Y-%m-%d")
task = "get the DB id for DevOps DB"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_SEARCH_NOTION_PAGE])

# First, let's search for your database
search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)

db_id = search_result[0]['data']['response_data']['results'][0]['id']
print("DB ID:", db_id)

task = f"get the page to do's in DB with id {db_id}"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_QUERY_DATABASE])

search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)



[2024-11-23 04:57:43,155][INFO] Executing `NOTION_SEARCH_NOTION_PAGE` with params={'query': 'DevOps DB', 'filter_value': 'database'} and metadata={} connected_account_id=None
[2024-11-23 04:57:46,528][INFO] Got response={'successfull': True, 'data': {'response_data': {'object': 'list', 'results': [{'object': 'database', 'id': '11b53e39-0487-804b-bca4-dd524767939b', 'cover': {'type': 'external', 'external': {'url': 'https://www.notion.so/images/page-cover/nasa...


DB ID: 11b53e39-0487-804b-bca4-dd524767939b


In [17]:
print(search_result)


[]


### Google Calendar

In [5]:

tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_CREATE_EVENT])

task = "Create a 1 hour meeting event at 5:30PM tomorrow regarding OpenAI Integration"

today = datetime.now().strftime("%Y-%m-%d")

response = openai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)
result = composio_toolset.handle_tool_calls(response)
print(result)


[2024-11-23 04:40:44,386][INFO] Executing `GOOGLECALENDAR_CREATE_EVENT` with params={'calendar_id': 'primary', 'create_meeting_room': True, 'event_duration': '1h', 'start_datetime': '2024-11-24T17:30:00', 'summary': 'OpenAI Integration Meeting'} and metadata={} connected_account_id=None
[2024-11-23 04:40:46,334][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#event', 'etag': '"3464673691676000"', 'id': 'i62bupprdrf2veopdjo7l7629k', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=aTYyYnVwcHJkcmYydmVvcGRq...


[{'successfull': True, 'data': {'response_data': {'kind': 'calendar#event', 'etag': '"3464673691676000"', 'id': 'i62bupprdrf2veopdjo7l7629k', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=aTYyYnVwcHJkcmYydmVvcGRqbzdsNzYyOWsgbWljaGFlbEBmbG93b24uYWk', 'created': '2024-11-23T04:40:45.000Z', 'updated': '2024-11-23T04:40:45.838Z', 'summary': 'OpenAI Integration Meeting', 'creator': {'email': 'michael@flowon.ai', 'self': True}, 'organizer': {'email': 'michael@flowon.ai', 'self': True}, 'start': {'dateTime': '2024-11-24T17:30:00Z', 'timeZone': 'UTC'}, 'end': {'dateTime': '2024-11-24T18:30:00Z', 'timeZone': 'UTC'}, 'iCalUID': 'i62bupprdrf2veopdjo7l7629k@google.com', 'sequence': 0, 'hangoutLink': 'https://meet.google.com/egu-qfzk-xcg', 'conferenceData': {'createRequest': {'requestId': '0a06f1a31bcb4b8fb75d8144fc6848fb', 'conferenceSolutionKey': {'type': 'hangoutsMeet'}, 'status': {'statusCode': 'success'}}, 'entryPoints': [{'entryPointType': 'video', 'uri': 'http